# Region growing segmentation

Based on: https://pcl.readthedocs.io/projects/tutorials/en/latest/region_growing_segmentation.html

With the PCL implementation, a point is picked with minimum curvature value and the region growth process is started. In our implementation, we start from initial seed points that are considered to be inside the object to be segmented. 

For every point in the seed list, the algorithm finds neighboring points. Every neighbor is tested for the angle between its normal and normal of the current seed point. If the angle is less than the threshold value then current point is added to the current region.

In [ ]:
# Add project src to path.
import set_path

# Import modules.
import numpy as np
import pylas

from src.cluster.region_growing import RegionGrowing
from src.utils.labels import Labels
from src.utils.las_utils import label_and_save_las

## Process a single file

In [ ]:
# Class with the initial seed points that we want to grow
seed_point_label = Labels.BUILDING
unlabelled_label = 1
# Select the file to process.
in_file = '../datasets/pointcloud/labelled_2386_9702.laz'
out_file = '../datasets/pointcloud/grown_2386_9702.laz'

# Read LiDAR point cloud
las = pylas.read(in_file)
# Initialize region growing class
reg = RegionGrowing()
# Process the file.
reg.set_input_cloud(las)
reg.set_initial_seed_points(las.label, seed_point_label)
regions = reg.region_growing()

In [ ]:
# Save classification labels to .laz file
label_and_save_las(las, regions, out_file)